In [ ]:
import pandas as pd
import numpy as np

csv_name = "original_data_by_municipality.csv"

df = pd.read_csv(csv_name, sep=",", encoding="ISO-8859-1")

df["Original_Order"] = np.arange(len(df))
df = df.sort_values(by=["Datum und Vorlage", "Original_Order"])
df = df.drop("Original_Order", axis=1)

print(df.head())

df.rename(
    columns={"Kanton (-) / Bezirk (>>) / Gemeinde (......)": "Gemeinde"}, inplace=True
)

df["Kanton"] = np.nan
df["Bezirk"] = np.nan

current_kanton = "Test Kanton"
current_bezirk = "Test Bezirk"

for index, row in df.iterrows():
    if row["Gemeinde"].startswith("-"):
        current_kanton = row["Gemeinde"][1:].strip()
    elif row["Gemeinde"].startswith(">>"):
        current_bezirk = row["Gemeinde"][2:].strip()
    elif row["Gemeinde"].startswith("......"):

        df.at[index, "Kanton"] = current_kanton
        df.at[index, "Bezirk"] = current_bezirk

gemeinde_df = df[df["Gemeinde"].str.startswith("......")].copy()

gemeinde_df["Gemeinde"] = gemeinde_df["Gemeinde"].map(lambda x: x[6:].strip())


gemeinde_df["Ja in %"] = pd.to_numeric(gemeinde_df["Ja in %"], errors="coerce")


pivot_ja = gemeinde_df.pivot_table(
    index=["Gemeinde", "Kanton", "Bezirk"],
    columns="Datum und Vorlage",
    values="Ja in %",
)

pivot_ja.to_csv("cleaned_data_by_municipality.csv")

print(df.head())

     Kanton (-) / Bezirk (>>) / Gemeinde (......)  \
491                                      - Zürich   
983                           >> Bezirk Affoltern   
1475                        ......Aeugst am Albis   
1967                     ......Affoltern am Albis   
2459                             ......Bonstetten   

                                      Datum und Vorlage Ja in %  
491   1960-05-29 Bundesbeschluss über die Weiterführ...   82.63  
983   1960-05-29 Bundesbeschluss über die Weiterführ...   77.72  
1475  1960-05-29 Bundesbeschluss über die Weiterführ...   70.93  
1967  1960-05-29 Bundesbeschluss über die Weiterführ...   77.27  
2459  1960-05-29 Bundesbeschluss über die Weiterführ...   79.26  
                      Gemeinde  \
491                   - Zürich   
983        >> Bezirk Affoltern   
1475     ......Aeugst am Albis   
1967  ......Affoltern am Albis   
2459          ......Bonstetten   

                                      Datum und Vorlage Ja in %  Kanton  \
491  